In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [1]:
%cd /content
!rm -r jawiOCR
!git clone https://github.com/mirzasyhm/jawiOCR.git
%cd /content/jawiOCR/craft/

/content
Cloning into 'jawiOCR'...
remote: Enumerating objects: 879, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 879 (delta 29), reused 34 (delta 15), pack-reused 829 (from 1)
Receiving objects: 100% (879/879), 102.97 MiB | 24.24 MiB/s, done.
Resolving deltas: 100% (442/442), done.
/content/jawiOCR/craft


In [2]:
# Step 1: Delete existing data directory if exists
!rm -rf data_root_dir/

# Step 2: Download the LMDB tar file
!wget https://huggingface.co/datasets/mirzasyhm/synthetic_jawi_images/resolve/main/data_sentence_loc12k_split.tar -O jawi.tar

# Step 3: Extract the tar file
!tar -xf jawi.tar

# Step 4: Rename the extracted folder to "data"
!mv data_sentence_loc12k_split data_root_dir/


--2025-06-12 13:08:03--  https://huggingface.co/datasets/mirzasyhm/synthetic_jawi_images/resolve/main/data_sentence_loc12k_split.tar
Resolving huggingface.co (huggingface.co)... 3.163.189.114, 3.163.189.90, 3.163.189.37, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.114|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/f9/16/f916668cb9026f1a501d38e183f8b89b65d2f31a4159ab1330ade73ab5a5fc1e/61650e847750dbb049e3cac4619e01fbb5a4274a06ee4b64e82a9dad6b48785d?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27data_sentence_loc12k_split.tar%3B+filename%3D%22data_sentence_loc12k_split.tar%22%3B&response-content-type=application%2Fx-tar&Expires=1749737283&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTczNzI4M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2Y5LzE2L2Y5MTY2NjhjYjkwMjZmMWE1MDFkMzhlMTgzZjhiODliNjVkMmYzMWE0MTU5YWIxMzMwYWRlNzNhYjVhNWZjMWUv

In [3]:
!mkdir -p pretrained
# download the checkpoint into exp/
!wget -O pretrained/CRAFT_clr_amp_29500.pth \
  "https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/CRAFT_clr_amp_29500.pth"

--2025-06-12 13:08:34--  https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/CRAFT_clr_amp_29500.pth
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.90, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a4/0e/a40edb5fee9aa56b081d76cebeb85749da8b1935a5194f415623e5924b1f8594/907662057e401ba8725fba171bb7bbb7c2160b43d6f3a27d8c32d509196c8dca?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27CRAFT_clr_amp_29500.pth%3B+filename%3D%22CRAFT_clr_amp_29500.pth%22%3B&Expires=1749737314&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTczNzMxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E0LzBlL2E0MGVkYjVmZWU5YWE1NmIwODFkNzZjZWJlYjg1NzQ5ZGE4YjE5MzVhNTE5NGY0MTU2MjNlNTkyNGIxZjg1OTQvOTA3NjYyMDU3ZTQwMWJhODcyNWZiYTE3MWJiN2JiYjdjMjE2MGI0M2Q2ZjNhMjdkOGMzMm

In [4]:
!pip install pip-tools
!pip install -r requirements.txt

In [5]:
!CUDA_VISIBLE_DEVICES=0 python3 train.py --yaml=custom_data_train

Traceback (most recent call last):
  File "/content/jawiOCR/craft/train.py", line 16, in <module>
    from data.dataset import SynthTextDataSet, CustomDataset
  File "/content/jawiOCR/craft/data/dataset.py", line 25, in <module>
    from utils.util import saveInput, saveImage
  File "/content/jawiOCR/craft/utils/util.py", line 8, in <module>
    from crnn.utils import craft_utils
ModuleNotFoundError: No module named 'crnn'


In [6]:
!ls exp/custom_data_train/

ls: cannot access 'exp/custom_data_train/': No such file or directory


In [8]:
!python demo.py \
    --trained_model ./exp/custom_data_train/CRAFT_clr_amp_12000.pth \
    --image_path ./demo/demo_5.jpg \
    --output_dir ./my_demo_results/ \
    --text_threshold 0.7 \
    --link_threshold 0.4 \
    --low_text 0.4 \
    --poly

CUDA is available. Using: Tesla T4
Loading weights from checkpoint: ./exp/custom_data_train/CRAFT_clr_amp_12000.pth
/content/jawiOCR/craft/demo.py:303: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this

In [9]:
from google.colab import files


files.download('./my_demo_results/res_demo_5.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Mount Google Drive
import shutil
from google.colab import drive
drive.mount('/content/drive')

# Zip the outputs folder
shutil.make_archive('exp', 'zip', 'exp')

# Copy zipped file to Google Drive
shutil.copy('exp.zip', '/content/drive/MyDrive/exp3000.zip')

In [5]:
!mkdir -p ./exp/custom_data_train
# download the checkpoint into exp/
!wget -O ./exp/custom_data_train/CRAFT_clr_amp_12000.pth \
  "https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/CRAFT_clr_amp_12000.pth"

--2025-06-12 13:08:45--  https://huggingface.co/datasets/mirzasyhm/checkpoint_jawi/resolve/main/CRAFT_clr_amp_12000.pth
Resolving huggingface.co (huggingface.co)... 3.163.189.74, 3.163.189.90, 3.163.189.114, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/a4/0e/a40edb5fee9aa56b081d76cebeb85749da8b1935a5194f415623e5924b1f8594/d4a890d8272c45581195e9fe07f9f722b3f4023d1a16bd8b94f780796c2ed067?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27CRAFT_clr_amp_12000.pth%3B+filename%3D%22CRAFT_clr_amp_12000.pth%22%3B&Expires=1749737325&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0OTczNzMyNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2E0LzBlL2E0MGVkYjVmZWU5YWE1NmIwODFkNzZjZWJlYjg1NzQ5ZGE4YjE5MzVhNTE5NGY0MTU2MjNlNTkyNGIxZjg1OTQvZDRhODkwZDgyNzJjNDU1ODExOTVlOWZlMDdmOWY3MjJiM2Y0MDIzZDFhMTZiZDhiOTRmNz

In [7]:
!git pull

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 5 (delta 3), reused 5 (delta 3), pack-reused 0 (from 0)
Unpacking objects: 100% (5/5), 59.73 KiB | 3.98 MiB/s, done.
From https://github.com/mirzasyhm/jawiOCR
   90ed495..e0b8503  main       -> origin/main
Updating 90ed495..e0b8503
Fast-forward
 craft/demo/demo_5.jpg | Bin 0 -> 61127 bytes
 1 file changed, 0 insertions(+), 0 deletions(-)
 create mode 100644 craft/demo/demo_5.jpg
